In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

# the function responsible for generating the CAMs (Class Activation Maps)

In [ ]:
def gen_heatmap_img(img, model0, layer_name='last_conv_layer',viz_img=None,orig_img=None):
    preds_raw = model0.predict(img[np.newaxis])
    preds = preds_raw > 0.5
    class_idx = (preds.astype(int).sum(axis=1) - 1)[0]
    class_output_tensor = model0.output[:, class_idx]
    
    viz_layer = model0.get_layer(layer_name)
    grads = K.gradients(
                        class_output_tensor ,
                        viz_layer.output
                        )[0] 
    
    pooled_grads=K.mean(grads,axis=(0,1,2))
    iterate=K.function([model0.input],[pooled_grads, viz_layer.output[0]])
    
    pooled_grad_value, viz_layer_out_value = iterate([img[np.newaxis]])
    
    for i in range(pooled_grad_value.shape[0]):
        viz_layer_out_value[:,:,i] *= pooled_grad_value[i]
    
    heatmap = np.mean(viz_layer_out_value, axis=-1)
    heatmap = np.maximum(heatmap,0)
    heatmap /= np.max(heatmap)

    viz_img=cv2.resize(viz_img,(img.shape[1],img.shape[0]))
    heatmap=cv2.resize(heatmap,(viz_img.shape[1],viz_img.shape[0]))
    
    heatmap_color = cv2.applyColorMap(np.uint8(heatmap*255), cv2.COLORMAP_HOT)/255
    heated_img = heatmap_color*0.5 + viz_img*0.5
    

    print('raw output from model : ')
    print_pred(preds_raw)
    
    if orig_img is None:
        show_Nimages([img,viz_img,heatmap_color,heated_img])
    else:
        show_Nimages([orig_img,img,viz_img,heatmap_color,heated_img])

    plt.show()
    return heated_img,heatmap_color,heatmap

Here are simple tools to easily show images and prediction

In [ ]:
def show_image(image,figsize=None,title=None):
    
    if figsize is not None:
        fig = plt.figure(figsize=figsize)

    if image.ndim == 2:
        plt.imshow(image,cmap='gray')
    else:
        plt.imshow(image)
        
    if title is not None:
        plt.title(title)

def show_Nimages(imgs,scale=1):

    N=len(imgs)
    fig = plt.figure(figsize=(25/scale, 16/scale))
    for i, img in enumerate(imgs):
        ax = fig.add_subplot(1, N, i + 1, xticks=[], yticks=[])
        show_image(img)
        
def print_pred(array_of_classes):
    xx = array_of_classes
    s1,s2 = xx.shape
    for i in range(s1):
        for j in range(s2):
            print('%.3f ' % xx[i,j],end='')
        print('')

First let us test the first 5 test examples. For each test example, I show original input, a preprocessed input, heatmap and combined heatmap respectively.

In [ ]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm

%matplotlib inline

In [ ]:
import shutil
#shutil.rmtree("/kaggle/working/cropped")

In [ ]:
os.makedirs('/kaggle/working/cropped')

In [ ]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')

In [ ]:
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
model = tf.keras.models.load_model('../input/aptos-densenet-model/desenet_aptos_final.h5')

In [ ]:
def resize_image(image_path, desired_size=224):
    im = Image.open(image_path)
    im = im.resize((desired_size, desired_size ), resample=Image.LANCZOS)
    
    return im

In [ ]:
def gen_heatmap_img(img, model0, layer_name='last_conv_layer',viz_img=None,orig_img=None):
    preds_raw = model0.predict(img[np.newaxis])
    preds = preds_raw > 0.5
    class_idx = (preds.astype(int).sum(axis=1) - 1)[0]
    class_output_tensor = model0.output[:, class_idx]
    
    viz_layer = model0.get_layer(layer_name)
    grads = K.gradients(
                        class_output_tensor ,
                        viz_layer.output
                        )[0] 
    
    pooled_grads=K.mean(grads,axis=(0,1,2))
    iterate=K.function([model0.input],[pooled_grads, viz_layer.output[0]])
    
    pooled_grad_value, viz_layer_out_value = iterate([img[np.newaxis]])
    
    for i in range(pooled_grad_value.shape[0]):
        viz_layer_out_value[:,:,i] *= pooled_grad_value[i]
    
    heatmap = np.mean(viz_layer_out_value, axis=-1)
    heatmap = np.maximum(heatmap,0)
    heatmap /= np.max(heatmap)

    viz_img=cv2.resize(viz_img,(img.shape[1],img.shape[0]))
    heatmap=cv2.resize(heatmap,(viz_img.shape[1],viz_img.shape[0]))
    
    heatmap_color = cv2.applyColorMap(np.uint8(heatmap*255), cv2.COLORMAP_HOT)/255
    heated_img = heatmap_color*0.5 + viz_img*0.5
    
    return heated_img,heatmap_color,heatmap

In [ ]:
def load_image_ben_orig(path,resize=True,crop=False,norm255=True,keras=False):
    image = cv2.imread(path)
    
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        
    image=cv2.addWeighted( image,4, cv2.GaussianBlur( image , (0,0) ,  10) ,-4 ,128)
    
    if norm255:
        return image/255
    elif keras:
        
        image = np.expand_dims(image, axis=0)
        return preprocess_input(image)[0]
    else:
        return image.astype(np.int16)
    
    return image

def transform_image_ben(img,resize=True,crop=False,norm255=True,keras=False):  
    image=cv2.addWeighted( img,4, cv2.GaussianBlur( img , (0,0) ,  10) ,-4 ,128)
    
    if norm255:
        return image/255
    elif keras:
        image = np.expand_dims(image, axis=0)
        return preprocess_input(image)[0]
    else:
        return image.astype(np.int16)
    
    return image

In [ ]:
layer_name = 'relu' #'conv5_block16_concat'


In [ ]:


NUM_SAMP=3662
start_index=0
DATA_PATH = '../input/aptos2019-blindness-detection'
TRAIN_IMG_PATH = os.path.join(DATA_PATH, 'train_images')
layer_name = 'relu' #'conv5_block16_concat'
for i, (idx, row) in enumerate(train_df[:NUM_SAMP].iterrows()):
    index = idx
    index += start_index
    path = os.path.join(TRAIN_IMG_PATH, train_df['id_code'][index]+'.png')
    ben_img = load_image_ben_orig(path)
    input_img = np.empty((1,224, 224, 3), dtype=np.uint8)
    input_img[0,:,:,:] = resize_image(path)
        
    [heatmaped, heatmap_col,heatmap] = gen_heatmap_img(input_img[0],
                            model, layer_name=layer_name,viz_img=ben_img)
    img_rect = cv2.imread(path)
    img_rect = cv2.resize(img_rect, (224,224))
    img_rect = cv2.cvtColor(img_rect, cv2.COLOR_BGR2RGB)


    img_test = heatmap_col
    img_test=img_test.astype(np.uint8)

    img_t = cv2.cvtColor(img_test, cv2.COLOR_RGB2GRAY)

    ret, binary = cv2.threshold(img_t, 0, 255, cv2.THRESH_BINARY)


    contours, h = cv2.findContours(
         binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    
    a = np.zeros(len(contours)*2)
    b= np.zeros(len(contours)*2)
    j=0
    for c in contours:
        if len(contours)<2:
            x, y, w, h = cv2.boundingRect(c)
     # draw a green rectangle to visualize the bounding rect
            cv2.rectangle( img_test, (x, y), (x+w, y+h), (0, 255, 0), 1)
            cv2.rectangle( img_rect, (x, y), (x+w, y+h), (0, 255, 0), 1)
            cropped_image = img_rect[y:y+h, x:x+w]
            cropped_image = cv2.resize(cropped_image, (224,224))
        else:
            x, y, w, h = cv2.boundingRect(c)
     # draw a green rectangle to visualize the bounding rec
            a[j]=x
            b[j]=y
            j=j+1
            a[j]=x+w
            b[j]=y+h
        j=j+1

    if len(contours)<2:
        cropped_image = img_rect[y:y+h, x:x+w]
        cropped_image = cv2.resize(cropped_image, (224,224))
    else:
        x1=int(max(a))
        y1=int(max(b))
        x2=int(min(a))
        y2=int(min(b))
        w1=x1-x2
        h1=y1-y2
        cv2.rectangle( img_rect, (x2, y2), (x2+w1, y2+h1), (0, 255, 0), 1)

        cropped_image = img_rect[y2:y2+h1, x2:x2+w1]
        cropped_image = cv2.resize(cropped_image, (224,224))
        
        
    filename = '/kaggle/working/cropped/'+'cropped' + str(i) + '.jpg'

    cv2.imwrite(filename, cropped_image)
    